In [1]:
import nltk

import numpy
import numpy as np

import pandas
import pandas as pd

import sklearn
import sklearn as sk

import cssfeature
import csspipe
import csstransformer

import helper

import sqlite3


## Load data
(run bash ./src/init.sh first!)

In [2]:
#data = pd.read_csv("../raw_data/data_n.csv", parse_dates=True, infer_datetime_format=True)
#help(data.head)
#data.head(1)
db_path = './data.sqlite3'
sqlite_connection = sqlite3.connect(db_path)
data_reader = helper.CNeuReader(sqlite_connection)

data = data_reader.get_results()
data.head(3)


,sentence,cNeu
0,likes the sound of thunder.,True
1,is so sleepy it's not even funny that's she ca...,True
2,is sore and wants the knot of muscles at the b...,True


# Create DataFrame with string-length-featues

In [3]:

print(data.head(2))

## create length of sentence feature
sl_data = cssfeature.CharacterFeatures.string_length(data, column=0, column_name='sentence_length')

print(data.head(2))
print()
print(sl_data.head(2))



                                            sentence  cNeu
0                        likes the sound of thunder.  True
1  is so sleepy it's not even funny that's she ca...  True
                                            sentence  cNeu
0                        likes the sound of thunder.  True
1  is so sleepy it's not even funny that's she ca...  True

  sentence_length  cNeu
0              27  True
1              63  True


In [4]:
features = sl_data[['sentence_length']].astype(str)
labels = sl_data[['cNeu']].astype(bool)

print(features.head(2))
print(labels.head(2))

  sentence_length
0              27
1              63
   cNeu
0  True
1  True


In [5]:

split = sk.cross_validation.train_test_split(features, labels, train_size = 0.66, random_state= 6432119)

train_feat, test_feat, train_label, test_label = split

"""
print("train_feat\n%s\n" %train_feat.head(1))
print("train_label\n%s\n" %train_label.head(1))
print("test_feat\n%s\n" %test_feat.head(1))
print("test_label\n%s\n" %test_label.head(1))
"""


'\nprint("train_feat\n%s\n" %train_feat.head(1))\nprint("train_label\n%s\n" %train_label.head(1))\nprint("test_feat\n%s\n" %test_feat.head(1))\nprint("test_label\n%s\n" %test_label.head(1))\n'

# Create pipefactory

In [6]:
pipefactory = csspipe.PipeFactory()

### Create classifiers for testing

In [7]:

multinomial_nb = pipefactory.classifiers.multinomial_nb
bernoulli_nb = pipefactory.classifiers.bernoulli_nb

svc = pipefactory.classifiers.svc
linear_svc = pipefactory.classifiers.linear_svc

random_forest = pipefactory.classifiers.random_forest_classifier
k_neighbors = pipefactory.classifiers.k_neighors_classifier


## Create extractors for testing

In [8]:

tfidf_vectorizer = pipefactory.extractors.tfidf_vectorizer


## Create selectors for testing

In [9]:
k_best = pipefactory.selectors.select_k_best

In [14]:
import nltk
from nltk import tag
from nltk.corpus import treebank

postagger = nltk.tag.ClassifierBasedPOSTagger(train=treebank.tagged_sents())



In [24]:
mytagger = csstransformer.PartOfSpeech(ignore_unknown=True)

In [33]:
#postagger.evaluate(treebank.tagged_sents())
input = data.sentence[:5,]
print()
print(postagger.tag_sents(input))
print()
print(mytagger.transform(input))


[[('l', 'RB'), ('i', 'PRP'), ('k', 'VBP'), ('e', 'RB'), ('s', 'RBR'), (' ', '#'), ('t', 'JJS'), ('h', 'PDT'), ('e', 'WP$'), (' ', '#'), ('s', 'RBR'), ('o', 'WP'), ('u', 'PDT'), ('n', 'WRB'), ('d', 'PDT'), (' ', '#'), ('o', 'WP'), ('f', 'PDT'), (' ', '#'), ('t', 'RBS'), ('h', 'PDT'), ('u', 'FW'), ('n', 'WRB'), ('d', 'PDT'), ('e', 'WP$'), ('r', 'RBR'), ('.', '.')], [('i', 'PRP'), ('s', 'VBD'), (' ', '#'), ('s', 'PRP$'), ('o', 'WP'), (' ', '#'), ('s', 'VBZ'), ('l', 'PDT'), ('e', 'EX'), ('e', 'WP$'), ('p', 'RP'), ('y', 'WRB'), (' ', '#'), ('i', 'PRP'), ('t', 'VBP'), ("'", "''"), ('s', 'PRP$'), (' ', '#'), ('n', 'WRB'), ('o', 'WP'), ('t', 'RBS'), (' ', '#'), ('e', 'WP$'), ('v', 'WP$'), ('e', 'WP$'), ('n', 'WRB'), (' ', '#'), ('f', 'PDT'), ('u', 'FW'), ('n', 'WRB'), ('n', 'WRB'), ('y', 'PRP'), (' ', '#'), ('t', 'JJS'), ('h', 'PDT'), ('a', 'DT'), ('t', 'JJS'), ("'", 'POS'), ('s', 'NNS'), (' ', ':'), ('s', 'PRP$'), ('h', 'PDT'), ('e', 'RBR'), (' ', '#'), ('c', 'WP$'), ('a', 'DT'), ('n', 'JJ')

In [ ]:
## Pipelineing
# http://scikit-learn.org/stable/modules/pipeline.html
# http://zacstewart.com/2014/08/05/pipelines-of-featureunions-of-pipelines.html

class Test(object):
    
    def __init__(self):
        pass
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, input):
        return postagger.tag_sents(input)



raw_pipeline = pipefactory.create_pipe([
        #tfidf_vectorizer
        ('tfidf', sklearn.feature_extraction.text.TfidfVectorizer(ngram_range=(1,2), analyzer='char'))
        #, ('kNN', sklearn.neighbors.KNeighborsClassifier())
        , multinomial_nb
    ])

pos_pipeline = pipefactory.create_pipe([
        #tfidf_vectorizer
        ('part of speech', Test())
        #('part of speech', csstransformer.PartOfSpeech(ignore_unknown=True))
        , ('tfidf', sklearn.feature_extraction.text.TfidfVectorizer(ngram_range=(1,2), analyzer='char'))
        #, ('kNN', sklearn.neighbors.KNeighborsClassifier())
        , multinomial_nb
    ])

data = helper.CNeuReader(sqlite_connection).get_results()
#print(data.head())

#print(data['sentence'].astype(str).values)
#print(data['cNeu'].astype(str).values)

#help(pipeline.fit(data['sentence'].astype(str).values, data['cNeu'].astype(bool).values))
#pipeline.fit(data['sentence'].astype(str).values, data['cNeu'].astype(bool).values)


r1 = sk.cross_validation.cross_val_score(raw_pipeline, data['sentence'].astype(str), data['cNeu'].astype(bool), cv=10)
r2 = sk.cross_validation.cross_val_score(pos_pipeline, data['sentence'].astype(str), data['cNeu'].astype(bool), cv=10)

print(r1.mean())
print(r2.mean())

print(pos_pipeline)


In [125]:
test_data = data['sentence'].astype(str).values[:3]

print("test_data:\n%s" % test_data)

tfidf_trans = sklearn.feature_extraction.text.TfidfVectorizer(ngram_range=(1,2))
#help(tfidf_trans)
tfidf_trans.fit(test_data)
#tfidf_trans.transform(test_data)

print(tfidf_trans.get_feature_names()[:10])

type(tfidf_trans.transform(data))
print(tfidf_trans.transform(["funny", "and"]).toarray())

#tfidf_trans.get_feature_names()

test_data:
['likes the sound of thunder.'
 "is so sleepy it's not even funny that's she can't get to sleep."
 "is sore and wants the knot of muscles at the base of her neck to stop hurting. On the other hand, YAY I'M IN ILLINOIS! <3"]
['and', 'and wants', 'at', 'at the', 'base', 'base of', 'can', 'can get', 'even', 'even funny']
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]]


In [81]:
tfidf_vectorizer = sk.feature_extraction.text.TfidfVectorizer()

pipeline = sk.pipeline.Pipeline([('tfidf', tfidf_vectorizer),
                     ('chi2', sk.feature_selection.SelectKBest(sk.feature_selection.chi2, k=1000)),
                     ('nb', sk.naive_bayes.MultinomialNB())
                    ])

#pipeline.fit(train_feat[["STATUS"]], train_label[["cNEU"]])
#data = train_feat.STATUS
#target = train_label.cNEU.apply(lambda x: x == 'y')

#foo = tfidf_vectorizer.fit_transform(data[:1], target[:1])
#print(foo)
#idf = foo.idf_

#print(len(idf))
#print(len(data))

#pipeline.fit(data.sentence, data.cNeu)
import sklearn
import sklearn.feature_extraction.text

data = helper.CNeuReader(sqlite_connection).get_results()

features = data.sentence
labels = data.cNeu

cv = sklearn.feature_extraction.text.CountVectorizer()

cv_fit = cv.fit(features, labels)
cv_transform_features = cv_fit.transform(features)


tfidftrans = sklearn.feature_extraction.text.TfidfTransformer()

tfidftrans_fit = tfidftrans.fit(cv_transform_features, labels)
tfidftrans_transform_features = tfidftrans_fit.transform(cv_transform_features)

#print(cv_transform_features)
#print(tfidftrans_transform_features)

skb = sk.feature_selection.SelectKBest(sk.feature_selection.chi2, k=1000)
mnb = sk.naive_bayes.MultinomialNB()



mnb_fit = mnb.fit(tfidftrans_transform_features, labels.astype(bool))

print(hasattr(mnb_fit, "fit"))
print(hasattr(mnb_fit, "transform"))
print(hasattr(mnb_fit, "fit_transform"))

pipe = sklearn.pipeline.Pipeline([
    ('cv', sklearn.feature_extraction.text.CountVectorizer())
    , ('tfidf_trans', sklearn.feature_extraction.text.TfidfTransformer())
    , ('nb', sk.naive_bayes.MultinomialNB())
])

sk.cross_validation.cross_val_score(pipe, features.astype(str), labels.astype(bool))



#print(tfidf_vectorizer.fit_transform(features, labels))
#tfidf = tfidf_vectorizer.fit_transform(features, labels)
#print(tfidf)

#print(tfidf[[9]])


select_k = sk.feature_selection.SelectKBest(sk.feature_selection.chi2, k=1000)
print('select_k has transform: %s' % hasattr(select_k, 'transform'))
print('select_k has fit: %s' % hasattr(select_k, 'fit'))
print('select_k has fit_transform: %s' % hasattr(select_k, 'fit_transform'))


#help(tfidf_vectorizer)

#result = sk.cross_validation.cross_val_score(pipeline, data[[0]], data[[1]])

#print(result)

True
False
False
select_k has transform: True
select_k has fit: True
select_k has fit_transform: True


In [84]:
pipe_factory = csspipe.PipeFactory()


In [85]:
tfidf_nb_pipe = pipe_factory.create_tfidf_nb()

In [86]:
tfidf_nb_pipe.steps[0]

('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
         stop_words=None, strip_accents=None, sublinear_tf=False,
         token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
         vocabulary=None))

In [89]:
result = sk.cross_validation.cross_val_score(tfidf_nb_pipe, data[[0]].values, data[[1]].values)
print(result)

AttributeError: 'numpy.ndarray' object has no attribute 'lower'